In [10]:
import csv
import math

import matplotlib.pyplot as plt
import numpy as np

from keras import applications
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input
from keras.models import load_model


import cv2
from matplotlib import pyplot as plt
from skimage import color
from skimage.transform import resize

from IPython.display import Audio
from IPython.display import display
import time

In [11]:
import numpy as np

print(len(np.load('pac_arrays/train_all.npy')))
print(len(np.load('pac_arrays/test_all.npy')))
print(len(np.load('pac_arrays/dev_all.npy')))

25740
8580
8580


In [12]:
import csv
with open("pac_guide.csv") as file:
    reader = csv.reader(file)
    i = 0
    image_paths = []
    camera_nums = []
    
    for row in reader:
        index = row[0]
        camera_num = row[1]
        label = row[2]
        filename = row[3]
        image_paths.append(filename)
        camera_nums.append(camera_num)
        #print(index, camera_num, label, filename)
print(len(image_paths)) # number of rows: 112036
print(len(camera_nums))

sub_dir_hash = {}
for i, sub_dir in enumerate(np.load('sub_dirs.npy')):
    sub_dir_hash[sub_dir] = i

print(sub_dir_hash)

112036
112036
{'11': 4, '08': 3, '39': 7, '52': 8, '63': 11, '15': 5, '62': 10, '23': 6, '04': 1, '06': 2, '02': 0, '72': 12, '59': 9}


In [13]:
def depth_map_to_image(depth_map):
    img = cv2.normalize(depth_map, depth_map, 0, 1, cv2.NORM_MINMAX)
    img = np.array(img * 255, dtype=np.uint8)
    orig_img = img
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.applyColorMap(img, cv2.COLORMAP_OCEAN)
    #img = resize(img, 224, 224)

    return orig_img, img

def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 13)) # Change to one-hot
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 13)) # Change to one-hot
            
            for i, index in enumerate(batch_indices):
                #img = image.load_img(image_paths[index], target_size=(224, 224))
                data = np.load(image_paths[int(index)])
                depth_map = data['x'].astype(np.float32)
                orig_img, ocean = depth_map_to_image(depth_map)
                ocean = resize(ocean, (224, 224), mode='constant')
                
                X[i, :, :, :] = ocean
                # Convert to 1 hot vector
                one_hot = np.zeros(13)
                camera_num = camera_nums[int(index)]
                camera_ind = sub_dir_hash[camera_num]
                one_hot[camera_ind] = 1
                Y[i,:] = one_hot
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)

In [9]:
input_tensor = Input(shape=(224,224,3))
model = applications.VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512]
	 [[Node: block5_conv2_1/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=4354988, _device="/job:localhost/replica:0/task:0/gpu:0"](block5_conv2_1/random_uniform/shape)]]

Caused by op 'block5_conv2_1/random_uniform/RandomUniform', defined at:
  File "/usr/local/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/site-packages/zmq/eventloop/io

In [6]:
# build a classifier model to put on top of the convolutional model
x = model.output
x = Flatten(input_shape=(model.output_shape[1:]))(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', kernel_initializer='glorot_uniform')(x)
x = Dense(13, activation='softmax', name='output', kernel_initializer='glorot_uniform')(x)

# add new classifier model on top of convolutional base
new_model = Model(model.input, x)

In [7]:
for layer in new_model.layers[:19]:
    layer.trainable = False

In [8]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [9]:
# RMSprop
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

In [ ]:
minibatch_size = 64

train_indices = np.load('pac_arrays/train_all.npy')
test_indices = np.load('pac_arrays/test_all.npy')

image_generator(train_indices, minibatch_size)

epochs = 30
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)

# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=image_generator(test_indices, minibatch_size),
    nb_val_samples=test_steps)

new_model.save('pca_camera_classification.hdf5')



/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., epochs=30, validation_data=<generator..., validation_steps=135, steps_per_epoch=403)`


Epoch 1/30
403/403 [==============================] - 541s - loss: 2.5337 - acc: 0.1592 - val_loss: 2.0921 - val_acc: 0.4570
Epoch 2/30
403/403 [==============================] - 523s - loss: 1.9244 - acc: 0.3772 - val_loss: 1.5910 - val_acc: 0.5397
Epoch 3/30
403/403 [==============================] - 524s - loss: 1.5248 - acc: 0.5431 - val_loss: 1.2450 - val_acc: 0.7464
Epoch 4/30
403/403 [==============================] - 523s - loss: 1.2705 - acc: 0.6381 - val_loss: 1.0644 - val_acc: 0.7668
Epoch 5/30
403/403 [==============================] - 523s - loss: 1.0933 - acc: 0.6943 - val_loss: 0.9105 - val_acc: 0.8040
Epoch 6/30
403/403 [==============================] - 523s - loss: 0.9689 - acc: 0.7296 - val_loss: 0.8570 - val_acc: 0.7750
Epoch 7/30
403/403 [==============================] - 523s - loss: 0.8747 - acc: 0.7601 - val_loss: 0.6851 - val_acc: 0.8687
Epoch 8/30
403/403 [==============================] - 523s - loss: 0.7992 - acc: 0.7810 - val_loss: 0.6578 - val_acc: 0.8665


In [33]:
time.sleep(2)
sound_file = 'http://www.pacdv.com/sounds/interface_sound_effects/sound82.wav'
display(Audio(url=sound_file, autoplay=True))
time.sleep(2)
display(Audio(url=sound_file, autoplay=True))

for i in range(0, 3):
    time.sleep(2)
    display(Audio(url=sound_file, autoplay=True))

In [5]:
new_model = load_model('pca_camera_classification.hdf5')


In [7]:
new_model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=1e-07, decay=0.0), metrics=['accuracy'])

In [8]:


minibatch_size = 64

train_indices = np.load('pac_arrays/train_all.npy')
test_indices = np.load('pac_arrays/test_all.npy')

image_generator(train_indices, minibatch_size)

epochs = 1
minibatch_size = 64

train_steps = math.ceil(len(train_indices) / minibatch_size)
test_steps = math.ceil(len(test_indices) / minibatch_size)



In [ ]:
# fine-tune the model
history = new_model.fit_generator(
    image_generator(train_indices, minibatch_size),
    steps_per_epoch=train_steps,
    epochs=epochs
    )

Epoch 1/1
348/403 [========================>.....] - ETA: 52s - loss: 0.3836 - acc: 0.8851

In [14]:

history = new_model.fit_generator(
    image_generator(test_indices, minibatch_size),
    steps_per_epoch=test_steps,
    epochs=epochs
    )


Epoch 1/1
135/135 [==============================] - 132s - loss: 0.3652 - acc: 0.8921   
